In [2]:
import clipper_admin

In [3]:
import fasttext

## making sure fasttext model loads and works properly

In [10]:
model = fasttext.load_model('fasttext_py_model.bin')

In [13]:
model.labels

[u'__label__1', u'__label__2']

In [14]:
model.loss_name

u'softmax'

In [15]:
model.predict(['had a nice experience'])

[[u'__label__2']]

In [16]:
model.predict(['could have been better'])

[[u'__label__1']]

In [25]:
reviews = ['so nice', 'very happy', 'meh']
fasttext_predict(reviews)

["[u'__label__2']", "[u'__label__2']", "[u'__label__1']"]

## tried to get fasttext to serve predictions using the python_closure deployer in clipper, but I think because it expects a pickle file it didnt't work. 

In [24]:
def fasttext_predict(reviews):
    """expects reviews passed as a values in a list"""
    preds = model.predict(reviews)
    return [str(p) for p in preds]

In [54]:
from clipper_admin import ClipperConnection, DockerContainerManager

In [55]:
clipper_conn = ClipperConnection(DockerContainerManager())

## starting clipper always works fine.. each container for query frontend, management backend and redis database starts fine

In [56]:
clipper_conn.start_clipper()

18-02-16:20:34:09 INFO     [docker_container_manager.py:96] Starting managed Redis instance in Docker
18-02-16:20:34:11 INFO     [clipper_admin.py:109] Clipper still initializing.
18-02-16:20:34:12 INFO     [clipper_admin.py:111] Clipper is running


## also allows us to register an app at the appropriate localhost

In [76]:
clipper_conn.register_application(name="fasttext-predict", input_type="string", default_output="none",
                                 slo_micros = 100000)

18-02-16:22:00:48 INFO     [clipper_admin.py:186] Application fasttext-predict was successfully registered


In [78]:
clipper_conn.get_all_apps()

[u'featuresum', u'fasttext-predict']

In [79]:
clipper_conn.get_app_info('fasttext-predict')

{u'default_output': u'none',
 u'input_type': u'strings',
 u'latency_slo_micros': 100000,
 u'linked_models': [],
 u'name': u'fasttext-predict'}

## trying to access the local host doesn't work though! what's going on??

In [77]:
clipper_conn.get_query_addr()

'localhost:1337'

In [72]:
clipper_conn.build_model("fasttext-predict", 1, "models/fasttext_py_model.bin",
                        "clipper/py-rpc:0.2",)

18-02-16:21:49:43 INFO     [clipper_admin.py:381] Building model Docker image with model data from models/fasttext_py_model.bin
18-02-16:21:51:58 INFO     [clipper_admin.py:385] Pushing model Docker image to fasttext-predict:1


'fasttext-predict:1'

In [84]:
clipper_conn.deploy_model("fasttext-predict", 1, "strings", "clipper/py-rpc:0.2/fasttext-predict:1")

18-02-16:22:32:24 INFO     [docker_container_manager.py:205] Found 0 replicas for fasttext-predict:1. Adding 1


APIError: 400 Client Error: Bad Request ("invalid reference format")